# 02a – 2. Experiment: Stage-2-Fine-Tuning: Erweitertes Training auf MCoRec

## Hypothese

BL4 (AV-HuBERT Cocktail, MCoRec-feingetunet) wurde bereits auf einem Multi-Corpus-Mix
aus LRS2, Vox2, AVYT, AVYT-mix und MCoRec trainiert (~400 k Schritte, LR 1·10⁻⁴).
Kann ein zweites, gezieltes Fine-Tuning (**Stage 2**) mit niedrigerer Lernrate
und weniger Schritten die Domänenanpassung weiter verbessern?

## Vorgehen

Stage-2-Checkpoint `avsr_cocktail_mcorec_stage2_lr5e-5_30k`:
- **Ausgangspunkt:** BL4-Checkpoint (`avsr_cocktail_mcorec_finetune`)
- **Lernrate:** 5·10⁻⁵ (halbiert gegenüber Stage 1)
- **Schritte:** 30 000 (statt 400 000) – moderates Nachtraining
- **Datenmix:** gleicher Streaming-Mix mit `--include_mcorec`

## Ergebnis (Vorschau)

Das Stage-2-Modell ist **konsistent schlechter** als BL4 (WER +0.036–0.037, JER +0.018).
Das zusätzliche Training hat das Modell überanpasst und von der besser generalisierenden
BL4-Lösung wegoptimiert. → Stage-2-Ansatz wird nicht weiterverfolgt.

**Hinweis zum Bugfix:** Dieser Lauf wurde **vor dem Bugfix** in `segmentation.py` durchgeführt (`min_duration_off` las fälschlicherweise den Wert von `min_duration_on`). Das ist **gewollt**: Der Bugfix wurde erst nach Abschluss der LLM- und Hyperparameter-Experimente entdeckt. Da der Bugfix allein die WER zunächst verschlechterte, wurde erst in `02j_`/`02k_` die Kombination aus Bugfix + `min_duration`-Optimierung erarbeitet, die schließlich das beste Ergebnis lieferte.

## 1 – Setup: Arbeitsverzeichnis & Imports

In [1]:
import os, sys
from pathlib import Path

# Pfad anpassen, falls nötig
project_baseline_path = Path("/home/josch080/Projektgruppe/mcorec_baseline")
os.chdir(project_baseline_path)
print("CWD:", os.getcwd())

# Repo-Root in sys.path aufnehmen, damit projektinterne Module importierbar sind
if str(project_baseline_path) not in sys.path:
    sys.path.append(str(project_baseline_path))


CWD: /home/josch080/Projektgruppe/mcorec_baseline


## 2 – GPU-Auswahl

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## 3 – CUDA-Verifikation

In [3]:
import torch
print("n_gpu:", torch.cuda.device_count())
# sollte 1 ausgeben


n_gpu: 1


## 4 – Stage-2-Training

Das Training wird als Subprocess gestartet. Die Argumente spiegeln das Stage-2-Setup
wider: niedrigere Lernrate, weniger Schritte, gleicher Datenmix wie Stage 1.

**Hinweis:** `subprocess.run(cmd)` ist bereits ausgeführt; der Checkpoint liegt unter
`model-bin/avsr_cocktail_mcorec_stage2_lr5e-5_30k/checkpoint-30000`.

In [4]:
cmd = [
    sys.executable, "script/train.py",
    "--streaming_dataset", # Daten werden on-the-fly geladen (kein vollständiges Preprocessing nötig)
    "--include_mcorec", # MCoRec-Daten in den Trainingsmix aufnehmen
    "--batch_size", "4",
    "--max_steps", "30000", # Weniger Schritte als Stage 1 (400k)
    "--gradient_accumulation_steps", "2", # Effektive Batch-Größe = 4 × 2 = 8
    "--save_steps", "2000", # Checkpoint alle 2000 Schritte sichern
    "--eval_steps", "2000", # Validation-Loss alle 2000 Schritte berechnen
    "--log_interval", "25",
    "--learning_rate", "5e-5", # Halbierte LR gegenüber Stage 1 (1e-4)
    "--warmup_steps", "3000", # Linearer LR-Warmup über die ersten 3000 Schritte
    "--checkpoint_name", "avsr_cocktail_mcorec_stage2_lr5e-5_30k", # Name des Ausgabe-Checkpoints
    "--model_name_or_path", "./model-bin/avsr_cocktail_mcorec_finetune", # Startpunkt: BL4
    "--output_dir", "./model-bin",
    "--report_to", "none", # Kein Logging an W&B o.ä.
]
print(" ".join(cmd)) # Vollständigen Befehl zur Verifikation ausgeben

/home/josch080/Projektgruppe/mcorec_train/bin/python script/train.py --streaming_dataset --include_mcorec --batch_size 4 --max_steps 30000 --gradient_accumulation_steps 2 --save_steps 2000 --eval_steps 2000 --log_interval 25 --learning_rate 5e-5 --warmup_steps 3000 --checkpoint_name avsr_cocktail_mcorec_stage2_lr5e-5_30k --model_name_or_path ./model-bin/avsr_cocktail_mcorec_finetune --output_dir ./model-bin --report_to none


In [5]:
import subprocess
# Training als Subprocess starten – Ausgabe erscheint direkt im Notebook
subprocess.run(cmd)

Loading pretrained model from ./model-bin/avsr_cocktail_mcorec_finetune
Loading MCoRec dataset
map_datasets
 {'lrs2': {'probabilities': 0.25, 'dataset': {'train': IterableDataset({
    features: ['label', 'length', 'sample_id', 'video'],
    num_shards: 10
}), 'valid': None}}, 'vox2': {'probabilities': 0.1, 'dataset': {'train': IterableDataset({
    features: ['label', 'length', 'sample_id', 'video'],
    num_shards: 53
}), 'valid': None}}, 'avyt': {'probabilities': 0.2, 'dataset': {'train': IterableDataset({
    features: ['label', 'length', 'sample_id', 'video'],
    num_shards: 16
}), 'valid': None}}, 'avyt-mix': {'probabilities': 0.25, 'dataset': {'train': IterableDataset({
    features: ['label', 'length', 'sample_id', 'video'],
    num_shards: 664
}), 'valid': None}}, 'mcorec': {'probabilities': 0.2, 'dataset': {'train': IterableDataset({
    features: ['label', 'length', 'sample_id', 'video'],
    num_shards: 48
}), 'valid': IterableDataset({
    features: ['label', 'length', 's

/home/josch080/Projektgruppe/mcorec_baseline/src/custom_trainer.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `AVSRTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
Could not estimate the number of tokens of the input, floating-point operations will not be computed
  0%|          | 25/30000 [00:19<4:11:33,  1.99it/s]

{'loss': 17.3312, 'grad_norm': 71.00940704345703, 'learning_rate': 3.5000000000000004e-07, 'epoch': 0.0}


  0%|          | 50/30000 [00:31<4:02:24,  2.06it/s]

{'loss': 14.2914, 'grad_norm': 15.612455368041992, 'learning_rate': 7.666666666666667e-07, 'epoch': 0.0}


...

{'loss': 22.8197, 'grad_norm': 49.814903259277344, 'learning_rate': 3.148148148148148e-08, 'epoch': 1.0}


Too many dataloader workers: 10 (max is dataset.num_shards=3). Stopping 7 dataloader workers.
100%|██████████| 30000/30000 [4:50:36<00:00,  2.31it/s]

{'eval_loss': 27.49915313720703, 'eval_runtime': 170.7197, 'eval_samples_per_second': 22.798, 'eval_steps_per_second': 5.699, 'epoch': 1.0}
{'train_runtime': 17445.3793, 'train_samples_per_second': 13.757, 'train_steps_per_second': 1.72, 'train_loss': 17.871614496358237, 'epoch': 1.0}


100%|██████████| 30000/30000 [4:50:45<00:00,  1.72it/s]


CompletedProcess(args=['/home/josch080/Projektgruppe/mcorec_train/bin/python', 'script/train.py', '--streaming_dataset', '--include_mcorec', '--batch_size', '4', '--max_steps', '30000', '--gradient_accumulation_steps', '2', '--save_steps', '2000', '--eval_steps', '2000', '--log_interval', '25', '--learning_rate', '5e-5', '--warmup_steps', '3000', '--checkpoint_name', 'avsr_cocktail_mcorec_stage2_lr5e-5_30k', '--model_name_or_path', './model-bin/avsr_cocktail_mcorec_finetune', '--output_dir', './model-bin', '--report_to', 'none'], returncode=0)

## 5 – Inference-Setup

Experiment-Utilities neu laden, da oben ggf. CWD geändert wurde.

In [1]:
import os, sys
import pandas as pd

# Arbeitsverzeichnis auf Repo-Root setzen (Voraussetzung für alle relativen Pfade)
project_baseline_path = "/home/josch080/Projektgruppe/mcorec_baseline"
os.chdir(project_baseline_path)

# Repo-Root in sys.path, damit projektinterne Module importierbar sind
if project_baseline_path not in sys.path:
    sys.path.append(project_baseline_path)

from script.pg_utils_experiments import run_inference_for_experiment, run_eval_and_log, append_eval_results_for_experiments

## 6 – Modell-Definitionen

Zwei Modelle im Vergleich: BL4 als Referenz und das neue Stage-2-Modell.
Der Checkpoint-Pfad zeigt auf den letzten gespeicherten Step (30 000).

In [2]:
MODELS = {
    # BL4: Referenzmodell aus vorherigen Experimenten
    "cocktail_finetuned": {
        "model_type": "avsr_cocktail",
        "chkpt": "model-bin/avsr_cocktail_mcorec_finetune",
        "out": "output_avsr_cocktail_finetuned",
    },
    # Stage-2-Modell: BL4 + 30k weiterer Trainingsschritte
    "cocktail_stage2": {
        "model_type": "avsr_cocktail",
        "chkpt": "model-bin/avsr_cocktail_mcorec_stage2_lr5e-5_30k/checkpoint-30000",
        "out": "output_avsr_cocktail_stage2",
    },
}

## 7 – Sessions & Experimente

Gleiches 5-Session-Subset wie in `02_`. Drei Experimente testen das Stage-2-Modell
mit den Baseline-Parametern (E11) und den bisher besten Parametern (E12, E13).

In [3]:
# Gleiches Dev-Subset wie in 02_ für direkte Vergleichbarkeit
SESSION_IDS = ["session_40", "session_43", "session_49", "session_50", "session_54"]

In [4]:
EXPERIMENTS = {
    "E11_stage2_bs3_len15": {
        "base_model": "cocktail_stage2",
        "beam_size": 3,
        "max_length": 15,
        "comment": "Stage-2-Modell, beam=3, len=15",
    },
    "E12_stage2_bs12_len20": {
        "base_model": "cocktail_stage2",
        "beam_size": 12,
        "max_length": 20,
        "comment": "Stage-2-Modell, beam=12, len=20 (beste Konfiguration)",
    },
    "E13_stage2_bs8_len20": {
        "base_model": "cocktail_stage2",
        "beam_size": 8,
        "max_length": 20,
        "comment": "Stage-2-Modell, beam=8, len=20",
    },
}


## 8 – Inference

3 Experimente × 5 Sessions = 15 Läufe. Ergebnisse liegen bereits vor.

In [26]:
for sid in SESSION_IDS:
    session_dir = f"data-bin/dev/{sid}"
    print(f"\n########## Starte Experimente für {sid} ##########")

    for exp_name in EXPERIMENTS:
        run_inference_for_experiment(
            exp_name=exp_name,
            base_models=MODELS,
            experiments=EXPERIMENTS,
            session_dir=session_dir,
        )


########## Starte Experimente für session_40 ##########

Starte Inference für Experiment: E11_stage2_bs3_len15
  base_model      = cocktail_stage2
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_stage2_lr5e-5_30k/checkpoint-30000
  beam_size       = 3
  max_length      = 15
  output_dir_name = output_E11_stage2_bs3_len15
  session_dir     = data-bin/dev_without_central_videos/dev/session_40
  comment         = Stage-2-Modell, beam=3, len=15
Loading avsr_cocktail model...
Loading model from model-bin/avsr_cocktail_mcorec_stage2_lr5e-5_30k/checkpoint-30000
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_40


Processing speakers:   0%|          | 0/6 [00:00<?, ?it/s]


Processing speakers:  17%|█▋        | 1/6 [00:36<03:01, 36.24s/it]


Processing speakers:  33%|███▎      | 2/6 [01:48<03:49, 57.31s/it]


Processing speaker spk_2 track 1 of 3: 0it [00:00, ?it/s]

Processing speaker spk_2 track 2 of 3: 100%|██████████| 14/14 [00:38<00:00,  2.78s/it]

Processing speakers:  50%|█████     | 3/6 [02:59<03:11, 63.84s/it]

...


Processing speaker spk_0 track 1 of 2: 100%|██████████| 26/26 [00:47<00:00,  1.82s/it]

Processing speakers:  20%|██        | 1/5 [01:01<04:06, 61.58s/it]


Processing speakers:  40%|████      | 2/5 [02:19<03:33, 71.28s/it]


Processing speakers:  60%|██████    | 3/5 [03:15<02:08, 64.02s/it]


Processing speakers:  80%|████████  | 4/5 [04:40<01:12, 72.33s/it]


Processing speakers: 100%|██████████| 5/5 [05:25<00:00, 65.09s/it]


## 9 – Evaluation & Aggregation

In [5]:
# Ergebnisse aller Experimente auswerten und an die gemeinsame CSV anhängen
df_dev = append_eval_results_for_experiments(
    experiments=EXPERIMENTS,
    session_ids=SESSION_IDS,
    target_csv="results_dev_subset_by_session.csv",
)



########## Evaluate für session_40 ##########
Starte Evaluate: /home/josch080/Projektgruppe/mcorec_train/bin/python script/evaluate.py --session_dir data-bin/dev_without_central_videos/dev/session_40 --output_dir_name output_ --label_dir_name labels
Evaluating 1 sessions

=== Evaluating session session_40 ===

--- Evaluating output dir: output_E01_bs4_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.564, 'spk_1': 0.4281, 'spk_2': 0.5576, 'spk_3': 0.4283, 'spk_4': 0.4793, 'spk_5': 0.4189}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 1.0, 'spk_3': 1.0, 'spk_4': 1.0, 'spk_5': 1.0}
Joint ASR-Clustering Error Rate: {'spk_0': 0.282, 'spk_1': 0.21405, 'spk_2': 0.2788, 'spk_3': 0.21415, 'spk_4': 0.23965, 'spk_5': 0.20945}

--- Evaluating output dir: output_E02_bs8_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.561, 'spk_1': 0.4312, 'spk_2': 0.5506, 'spk_3': 0.4283, 'spk_4': 0.5041, 'spk_5': 0.4189}
Speaker clusterin

## 10 – Rohergebnisse (pro Session)

Vollständige Tabelle aller Stage-2-Experimente aufgeschlüsselt nach Session.

In [7]:
import pandas as pd

# gemeinsame Ergebnis-CSV laden
df = pd.read_csv("results_dev_subset_by_session.csv")

stage2_models = [
    "E11_stage2_bs3_len15",
    "E12_stage2_bs12_len20",
    "E13_stage2_bs8_len20",
]

# Nur Stage-2-Zeilen filtern und nach Modell + Session sortieren
stage2_df = (
    df[df["model"].isin(stage2_models)]
    .sort_values(["model", "session"])
)

display(stage2_df)


,session,exp,avg_conv_f1,avg_speaker_wer,avg_joint_error,model,timestamp
10,session_40,output_E11_stage2_bs3_len15,1.000000,0.554850,0.277425,E11_stage2_bs3_len15,2025-11-30T14:02:31
23,session_43,output_E11_stage2_bs3_len15,1.000000,0.559100,0.279550,E11_stage2_bs3_len15,2025-11-30T14:02:45
36,session_49,output_E11_stage2_bs3_len15,1.000000,0.473950,0.236975,E11_stage2_bs3_len15,2025-11-30T14:02:58
49,session_50,output_E11_stage2_bs3_len15,0.571429,0.576617,0.502608,E11_stage2_bs3_len15,2025-11-30T14:03:11
62,session_54,output_E11_stage2_bs3_len15,0.666667,0.572320,0.452820,E11_stage2_bs3_len15,2025-11-30T14:03:24
11,session_40,output_E12_stage2_bs12_len20,1.000000,0.529883,0.264942,E12_stage2_bs12_len20,2025-11-30T14:02:31
24,session_43,output_E12_stage2_bs12_len20,1.000000,0.543050,0.271525,E12_stage2_bs12_len20,2025-11-30T14:02:45
37,session_49,output_E12_stage2_bs12_len20,1.000000,0.464700,0.232350,E12_stage2_bs12_len20,2025-11-30T14:02:58
50,session_50,output_E12_stage2_bs12_len20,0.571429,0.569600,0.499100,E12_stage2_bs12_len20,2025-11-30T14:03:11
63,session_54,output_E12_stage2_bs12_len20,0.666667,0.548000,0.440660,E12_stage2_bs12_len20,2025-11-30T14:03:24


## 11 – Vergleich Stage-2 vs. BL4

Stage-2-Ergebnisse werden mit den BL4-Ergebnissen bei gleichen Hyperparametern verglichen.
Positive Δ-Werte bedeuten, dass Stage-2 *schlechter* als BL4 ist.

In [14]:
import pandas as pd

# CSV laden
dev_df = pd.read_csv("results_dev_subset_by_session.csv")

# Stage-2-Ergebnisse über alle Sessions mitteln
stage2_models = list(EXPERIMENTS.keys())
stage2_df = dev_df[dev_df["model"].isin(stage2_models)].copy()

stage2_by_model = (
    stage2_df
    .groupby("model")[["avg_conv_f1", "avg_speaker_wer", "avg_joint_error"]]
    .mean()
    .reset_index()
)

# Hyperparameter aus EXPERIMENTS-Dict in neue Spalten mappen
stage2_by_model["beam_size"] = stage2_by_model["model"].map(
    lambda m: EXPERIMENTS_STAGE2[m]["beam_size"]
)
stage2_by_model["max_length"] = stage2_by_model["model"].map(
    lambda m: EXPERIMENTS_STAGE2[m]["max_length"]
)

# Finetuned-Experimente (E01..E10) aggregieren
# Beam/Length direkt aus dem Modellnamen parsen.

ft_df = dev_df[dev_df["model"].str.startswith("E0")].copy()

def parse_beam_len(model_name: str):
    # Beispiel: "E08_bs8_len20" -> beam=8, length=20
    parts = model_name.split("_")
    beam = int(parts[1].replace("bs", ""))
    length = int(parts[2].replace("len", ""))
    return beam, length

# pd.Series erlaubt das Entpacken des Tupels in zwei Spalten per apply()
ft_df[["beam_size", "max_length"]] = ft_df["model"].apply(
    lambda m: pd.Series(parse_beam_len(m))
)

# BL4-Ergebnisse über Sessions mitteln
ft_agg = (
    ft_df
    .groupby(["model", "beam_size", "max_length"])[["avg_speaker_wer", "avg_joint_error"]]
    .mean()
    .reset_index()
)

# Stage-2 und BL4 per (beam_size, max_length) zusammenführen
comparison = pd.merge(
    stage2_by_model,
    ft_agg,
    on=["beam_size", "max_length"],
    how="left",
    suffixes=("_stage2", "_ft"),
)

# Δ-Spalten: positiv = Stage-2 schlechter als BL4
comparison["Δ_WER_stage2_minus_ft"] = (
    comparison["avg_speaker_wer_stage2"] - comparison["avg_speaker_wer_ft"]
)
comparison["Δ_JER_stage2_minus_ft"] = (
    comparison["avg_joint_error_stage2"] - comparison["avg_joint_error_ft"]
)

# Spalten hübsch sortieren
comparison_table = comparison[
    [
        "model_stage2", "beam_size", "max_length",
        "model_ft", "avg_speaker_wer_ft", "avg_joint_error_ft",
        "avg_speaker_wer_stage2", "avg_joint_error_stage2",
        "Δ_WER_stage2_minus_ft", "Δ_JER_stage2_minus_ft",
    ]
].sort_values(["max_length", "beam_size"])

print("Vergleich Stage-2 vs. finetuned (gleiche beam/max_length, aus results_dev_subset_by_session):")
display(comparison_table)


Vergleich Stage-2 vs. finetuned (gleiche beam/max_length, aus results_dev_subset_by_session):


,model_stage2,beam_size,max_length,model_ft,avg_speaker_wer_ft,avg_joint_error_ft,avg_speaker_wer_stage2,avg_joint_error_stage2,Δ_WER_stage2_minus_ft,Δ_JER_stage2_minus_ft
0,E11_stage2_bs3_len15,3,15,NaN,NaN,NaN,0.547367,0.349876,NaN,NaN
2,E13_stage2_bs8_len20,8,20,E08_bs8_len20,0.495798,0.324091,0.532463,0.342423,0.036665,0.018332
1,E12_stage2_bs12_len20,12,20,E09_bs12_len20,0.495416,0.323900,0.531047,0.341715,0.035631,0.017815


## 12 – Interpretation

| Konfiguration | WER BL4 | WER Stage-2 | Δ WER | JER BL4 | JER Stage-2 | Δ JER |
|---------------|---------|-------------|-------|---------|-------------|-------|
| beam=8,  len=20 | 0.4958 | 0.5325 | +0.037 | 0.3241 | 0.3424 | +0.018 |
| beam=12, len=20 | 0.4954 | 0.5310 | +0.036 | 0.3239 | 0.3417 | +0.018 |

Das Stage-2-Modell ist in beiden getesteten Konfigurationen konsistent **schlechter** als BL4.

**Erklärung:** BL4 wurde bereits umfangreich auf MCoRec optimiert. Das weitere Fine-Tuning
mit 30 000 Schritten und LR 5·10⁻⁵ hat das Modell überanpasst – es hat sich von der
besser generalisierenden Lösung wegbewegt, ohne neue Informationen zu lernen.

**Schlussfolgerung:** Stage-2-Fine-Tuning wird nicht weiterverfolgt. BL4 bleibt das
Referenzmodell für alle weiteren Experimente.
